In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'

In [6]:
def bon_bun_allocator(x):
    splitted = x.split('-')
    return int(splitted[0])

def bu_bun_allocator(x):
    splitted = x.split('-')
    if len(splitted) == 2:
        return int(splitted[1])
    elif len(splitted) == 1:
        return 0
    else:
        print('error')
        return

In [7]:
def land_plans_full_address_allocator(df):    
    bonbun = df['지번'].apply(bon_bun_allocator)
    bubun = df['지번'].apply(bu_bun_allocator)
    
    df['지번주소'] = df['법정동명'] + ' ' + bonbun.apply(str) + '-' + bubun.apply(str)
    
    return df

In [8]:
def land_plan_unstack(df):
    df['이진항'] = 1
    df = df.set_index(['전체주소', '년', '대장구분명', '용도지역지구명']).unstack(-1)
    
    return df

In [9]:
def create_land_plans_df():
    df = pd.read_csv('./토지이용계획정보/AL_11_D155_20201107.csv', encoding='euc-kr').drop_duplicates()
    df['년'] = df['등록일자'].str.split('-').apply(lambda x: x[0]).astype('int')
    
    df = land_plans_full_address_allocator(df)
    
    simple_cols = ['대장구분명', '저촉여부', '용도지역지구명', '년', '지번주소']
    
    df = df[simple_cols]
    
    df = df.drop_duplicates(subset=['대장구분명', '용도지역지구명', '년', '지번주소'], keep='first').reset_index(drop=True)
    
    df = land_plan_unstack(df)
    
    return df

In [10]:
df = create_land_plans_df()

KeyError: Index(['year'], dtype='object')

In [3]:
print(df.shape)
df.head()

(8455920, 13)


,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,도면번호,저촉여부코드,저촉여부,용도지역지구코드,용도지역지구명,등록일자,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,12900001111020080001UNE2010001001,1,포함,UNE201,대공방어협조구역(위탁고도:54-236m),2009-12-29,2020-06-09
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,13404001111020080001UOA1200010002,2,저촉,UOA120,상대보호구역,2009-12-28,2020-06-09
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQA01X0000001,1,포함,UQA01X,도시지역,2009-12-28,2020-06-09
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQF1100071000,2,저촉,UQF110,자연경관지구,2009-12-28,2020-06-09
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQS1000507000,2,저촉,UQS100,도로,2009-12-28,2020-06-09


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8455920 entries, 0 to 8455919
Data columns (total 14 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   고유번호          int64 
 1   법정동코드         int64 
 2   법정동명          object
 3   대장구분코드        int64 
 4   대장구분명         object
 5   지번            object
 6   도면번호          object
 7   저촉여부코드        int64 
 8   저촉여부          object
 9   용도지역지구코드      object
 10  용도지역지구명       object
 11  등록일자          object
 12  데이터기준일자       object
 13  split_length  int64 
dtypes: int64(5), object(9)
memory usage: 967.7+ MB


In [12]:
df.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,도면번호,저촉여부코드,저촉여부,용도지역지구코드,용도지역지구명,등록일자,데이터기준일자,split_length
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,12900001111020080001UNE2010001001,1,포함,UNE201,대공방어협조구역(위탁고도:54-236m),2009-12-29,2020-06-09,3
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,13404001111020080001UOA1200010002,2,저촉,UOA120,상대보호구역,2009-12-28,2020-06-09,3
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQA01X0000001,1,포함,UQA01X,도시지역,2009-12-28,2020-06-09,3
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQF1100071000,2,저촉,UQF110,자연경관지구,2009-12-28,2020-06-09,3
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQS1000507000,2,저촉,UQS100,도로,2009-12-28,2020-06-09,3


In [13]:
df.drop(columns=['split_length'], inplace=True)

In [14]:
df['year'] = df['등록일자'].apply(lambda x: x.split('-')[0])

In [15]:
df.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,도면번호,저촉여부코드,저촉여부,용도지역지구코드,용도지역지구명,등록일자,데이터기준일자,year
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,12900001111020080001UNE2010001001,1,포함,UNE201,대공방어협조구역(위탁고도:54-236m),2009-12-29,2020-06-09,2009
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,13404001111020080001UOA1200010002,2,저촉,UOA120,상대보호구역,2009-12-28,2020-06-09,2009
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQA01X0000001,1,포함,UQA01X,도시지역,2009-12-28,2020-06-09,2009
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQF1100071000,2,저촉,UQF110,자연경관지구,2009-12-28,2020-06-09,2009
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQS1000507000,2,저촉,UQS100,도로,2009-12-28,2020-06-09,2009


In [16]:
df.drop(columns=['등록일자', '데이터기준일자'], inplace=True)

In [17]:
df.drop_duplicates(inplace=True)

In [18]:
df.drop(columns=['용도지역지구코드', '고유번호', '법정동코드', '대장구분코드', '저촉여부코드', '도면번호'], inplace=True)

In [19]:
df.head()

,법정동명,대장구분명,지번,저촉여부,용도지역지구명,year
0,서울특별시 종로구 청운동,토지대장,1,포함,대공방어협조구역(위탁고도:54-236m),2009
1,서울특별시 종로구 청운동,토지대장,1,저촉,상대보호구역,2009
2,서울특별시 종로구 청운동,토지대장,1,포함,도시지역,2009
3,서울특별시 종로구 청운동,토지대장,1,저촉,자연경관지구,2009
4,서울특별시 종로구 청운동,토지대장,1,저촉,도로,2009


In [20]:
df['split_length'] = df['지번'].apply(lambda x: len(x.split('-')))
df['split_length'].value_counts()

2    7871689
1     584231
Name: split_length, dtype: int64

In [21]:
def landnum_modifier(x):
    splitted = x.split('-')
    if len(splitted) == 1:
        return x + '-0'
    else:
        return x

In [22]:
df['지번주소'] = df['법정동명'] + ' ' + df['지번'].apply(landnum_modifier)
df.head(20)

,법정동명,대장구분명,지번,저촉여부,용도지역지구명,year,split_length,지번주소
0,서울특별시 종로구 청운동,토지대장,1,포함,대공방어협조구역(위탁고도:54-236m),2009,1,서울특별시 종로구 청운동 1-0
1,서울특별시 종로구 청운동,토지대장,1,저촉,상대보호구역,2009,1,서울특별시 종로구 청운동 1-0
2,서울특별시 종로구 청운동,토지대장,1,포함,도시지역,2009,1,서울특별시 종로구 청운동 1-0
3,서울특별시 종로구 청운동,토지대장,1,저촉,자연경관지구,2009,1,서울특별시 종로구 청운동 1-0
4,서울특별시 종로구 청운동,토지대장,1,저촉,도로,2009,1,서울특별시 종로구 청운동 1-0
5,서울특별시 종로구 청운동,토지대장,1,포함,제1종일반주거지역,2009,1,서울특별시 종로구 청운동 1-0
6,서울특별시 종로구 청운동,토지대장,1,포함,가축사육제한구역,2011,1,서울특별시 종로구 청운동 1-0
7,서울특별시 종로구 청운동,토지대장,1,저촉,건축선,2019,1,서울특별시 종로구 청운동 1-0
8,서울특별시 종로구 청운동,토지대장,1,접함,자연녹지지역,2009,1,서울특별시 종로구 청운동 1-0
9,서울특별시 종로구 청운동,토지대장,1,포함,과밀억제권역,2014,1,서울특별시 종로구 청운동 1-0


In [23]:
df['대장구분명'].value_counts()

토지대장    8235648
임야대장     219903
Name: 대장구분명, dtype: int64

In [24]:
df.drop(columns=['법정동명', '지번'], inplace=True)
df.head()

,대장구분명,저촉여부,용도지역지구명,year,split_length,지번주소
0,토지대장,포함,대공방어협조구역(위탁고도:54-236m),2009,1,서울특별시 종로구 청운동 1-0
1,토지대장,저촉,상대보호구역,2009,1,서울특별시 종로구 청운동 1-0
2,토지대장,포함,도시지역,2009,1,서울특별시 종로구 청운동 1-0
3,토지대장,저촉,자연경관지구,2009,1,서울특별시 종로구 청운동 1-0
4,토지대장,저촉,도로,2009,1,서울특별시 종로구 청운동 1-0


In [26]:
df.drop(columns=['split_length'], inplace=True)

In [28]:
df.head()

,대장구분명,저촉여부,용도지역지구명,year,지번주소
0,토지대장,포함,대공방어협조구역(위탁고도:54-236m),2009,서울특별시 종로구 청운동 1-0
1,토지대장,저촉,상대보호구역,2009,서울특별시 종로구 청운동 1-0
2,토지대장,포함,도시지역,2009,서울특별시 종로구 청운동 1-0
3,토지대장,저촉,자연경관지구,2009,서울특별시 종로구 청운동 1-0
4,토지대장,저촉,도로,2009,서울특별시 종로구 청운동 1-0


In [74]:
prac_df = df.drop_duplicates(subset=['대장구분명', '용도지역지구명', 'year', '지번주소'], keep='first').reset_index(drop=True).copy()
print(df.shape)
print(prac_df.shape)
prac_df.head()

(8455920, 5)
(7946258, 5)


,대장구분명,저촉여부,용도지역지구명,year,지번주소
0,토지대장,포함,대공방어협조구역(위탁고도:54-236m),2009,서울특별시 종로구 청운동 1-0
1,토지대장,저촉,상대보호구역,2009,서울특별시 종로구 청운동 1-0
2,토지대장,포함,도시지역,2009,서울특별시 종로구 청운동 1-0
3,토지대장,저촉,자연경관지구,2009,서울특별시 종로구 청운동 1-0
4,토지대장,저촉,도로,2009,서울특별시 종로구 청운동 1-0


In [75]:
prac_df.set_index(['지번주소', '대장구분명', 'year', '용도지역지구명'], inplace=True)
print(prac_df.shape)
prac_df.head()

(7946258, 1)


저촉여부
지번주소              대장구분명 year 용도지역지구명                    
서울특별시 종로구 청운동 1-0 토지대장  2009 대공방어협조구역(위탁고도:54-236m)   포함
                             상대보호구역                   저촉
                             도시지역                     포함
                             자연경관지구                   저촉
                             도로                       저촉

In [76]:
unstack_df = prac_df.unstack(-1)
print(unstack_df.shape)
unstack_df.head()

(3174788, 329)


저촉여부                     \
용도지역지구명                             (한강)오염행위 제한지역 (한강)폐기물매립시설 설치제한지역   
지번주소                     대장구분명 year                                    
서울특별시 강남구 개포동 00000001-0 NaN   2014           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 00000002-0 NaN   2014           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 00000003-0 NaN   2014           NaN                NaN   

                                                                            \
용도지역지구명                             가로구역별 최고높이 제한지역 가스공급설비 가스공급시설 가축사육제한구역   
지번주소                     대장구분명 year                                          
서울특별시 강남구 개포동 00000001-0 NaN   2014             NaN    NaN    NaN      NaN   
                               2020             NaN    NaN    NaN      NaN   
서울특별시 강남구 개포동 00000002-0 NaN   2014             NaN    NaN    NaN      NaN   
                               2020             NaN    NaN    NaN      NaN   
서울특별시 강남구 개포동 00000003-0 NaN   2014             NaN    NaN    NaN      NaN   

                                                                            \
용도지역지구명                             개발제한구역 개발진흥지구 개발행위허가제한지역  건축선 건축용도지역기타   
지번주소                     대장구분명 year                                          
서울특별시 강남구 개포동 00000001-0 NaN   2014    NaN    NaN        NaN  NaN      NaN   
                               2020    NaN    NaN        NaN  NaN      NaN   
서울특별시 강남구 개포동 00000002-0 NaN   2014    NaN    NaN        NaN  NaN      NaN   
                               2020    NaN    NaN        NaN  NaN      NaN   
서울특별시 강남구 개포동 00000003-0 NaN   2014    NaN    NaN        NaN  NaN      NaN   

                                                                          \
용도지역지구명                             건축용도지역미분류 건축허가·착공제한지역 경관광장 경관녹지 경관지구   
지번주소                     대장구분명 year                                        
서울특별시 강남구 개포동 00000001-0 NaN   2014       NaN         NaN  NaN  NaN  NaN   
                               2020       NaN         NaN  NaN  NaN  NaN   
서울특별시 강남구 개포동 00000002-0 NaN   2014       NaN         NaN  NaN  NaN  NaN   
                               2020       NaN         NaN  NaN  NaN  NaN   
서울특별시 강남구 개포동 00000003-0 NaN   2014       NaN         NaN  NaN  NaN  NaN   

                                                                             \
용도지역지구명                             경관지구기타 고가도로 고도지구 고등학교 고속철도  고압선 공간시설미분류   
지번주소                     대장구분명 year                                           
서울특별시 강남구 개포동 00000001-0 NaN   2014    NaN  NaN  NaN  NaN  NaN  NaN     NaN   
                               2020    NaN  NaN  NaN  NaN  NaN  NaN     NaN   
서울특별시 강남구 개포동 00000002-0 NaN   2014    NaN  NaN  NaN  NaN  NaN  NaN     NaN   
                               2020    NaN  NaN  NaN  NaN  NaN  NaN     NaN   
서울특별시 강남구 개포동 00000003-0 NaN   2014    NaN  NaN  NaN  NaN  NaN  NaN     NaN   

                                                                          \
용도지역지구명                             공공공지 공공도서관 공공문화체육시설미분류 공공시설구역 공공시설용지   
지번주소                     대장구분명 year                                        
서울특별시 강남구 개포동 00000001-0 NaN   2014  NaN   NaN         NaN    NaN    NaN   
                               2020  NaN   NaN         NaN    NaN    NaN   
서울특별시 강남구 개포동 00000002-0 NaN   2014  NaN   NaN         NaN    NaN    NaN   
                               2020  NaN   NaN         NaN    NaN    NaN   
서울특별시 강남구 개포동 00000003-0 NaN   2014  NaN   NaN         NaN    NaN    NaN   

                                                                               \
용도지역지구명                             공공주택지구 공공지원민간임대주택 공급촉진지구 공공청사  공동구 공설화장시설   
지번주소                     대장구분명 year                                             
서울특별시 강남구 개포동 00000001-0 NaN   2014    NaN               NaN  NaN  NaN    NaN   
                               2020    NaN 

In [77]:
unstack_df.head(100)

저촉여부                     \
용도지역지구명                             (한강)오염행위 제한지역 (한강)폐기물매립시설 설치제한지역   
지번주소                     대장구분명 year                                    
서울특별시 강남구 개포동 00000001-0 NaN   2014           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 00000002-0 NaN   2014           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 00000003-0 NaN   2014           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 00000013-0 토지대장  2014           NaN                NaN   
서울특별시 강남구 개포동 100-0      토지대장  2014           NaN                NaN   
                               2017           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 101-0      토지대장  2014           NaN                NaN   
                               2017           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 102-0      토지대장  2014           NaN                NaN   
                               2015           NaN                NaN   
                               2017           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 103-0      토지대장  2014           NaN                NaN   
                               2015           NaN                NaN   
                               2017           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 104-1      토지대장  2014           NaN                NaN   
                               2017           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 104-2      토지대장  2014           NaN                NaN   
                               2017           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 104-3      토지대장  2014           NaN                NaN   
                               2017           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 104-4      토지대장  2019           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 1042-15    토지대장  2014           NaN                NaN   
                               2017           NaN                NaN   
                               2019           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 105-0      토지대장  2014           NaN                NaN   
                               2017           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 1055-0     토지대장  2014           NaN                NaN   
                               2015           NaN                NaN   
                               2017           NaN                NaN   
                               2019           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 1055-11    토지대장  2014           NaN                NaN   
                               2017           NaN                NaN   
                               2019           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 1055-12    토지대장  2014           NaN                NaN   
                               2015           NaN                NaN   
                               2017           NaN                NaN   
                               2020           NaN                NaN   
서울특별시 강남구 개포동 1055-14    토지대장  2014           NaN                NaN   
             